In [ ]:
 # uncomment following for interactive matplotlib
# %matplotlib notebook

from svm2 import *
import scipy.io
import numpy as np
import matplotlib.pyplot as plt

# Part 1: Kernel SVM
## Test on trivial data

In [ ]:
X = np.array([[1, 2, 1, -1, -1, -2], [1, 1, 2, -1, -2, -1]])
y = np.array([[1, 1, 1, -1, -1, -1]])

### get_kernel function

In [ ]:
K = get_kernel(X, X, {'kernel': 'linear'})
print(K)

In [ ]:
K = get_kernel(X, X, {'kernel': 'polynomial', 'd': 2})
print(K)

In [ ]:
K = get_kernel(X, X, {'kernel': 'rbf', 'sigma': 1.0})
np.set_printoptions(precision=4, suppress=True)
print(K)
np.set_printoptions()

### my_kernel_svm function

In [ ]:
y = np.array([[-1, 1, 1, 1, -1, -1]])
C = float('inf');
options = {'verb': True, 'tmax': float('inf'),
           'kernel': 'rbf', 'sigma': 0.01}

In [ ]:
model = my_kernel_svm(X, y, C, options)
print(model)

In [ ]:
classif = classif_kernel_svm(X, model)
print(classif)

In [ ]:
plot_pts(X, y)
plt.axis('equal')
plt.grid(linestyle='--')
plt.xlim(-3, 3)
plt.ylim(-3, 3)
plot_boundary(plt.gca(), model)


## Test on flower

In [ ]:
 def unwrap(data):
    """
    Simple "hack" for preparing data from *.mat files
    """
    try:
        while (len(data) == 1) and (len(data.shape) > 0):
            data = data[0]
        for key in list(data.dtype.names):
            data[key] = unwrap(data[key])
    except:
        pass
    return data

def ndarray2dict(data, indexes=None):
    outputs = {}
    for key in list(data.dtype.names):
        value = unwrap(data[key])
        try:
            if len(value.shape) > 0:
                value = np.atleast_2d(value)
        except:
            pass
        outputs[key] = value
    if indexes is not None:
        for key in indexes:
            outputs[key] -= 1
    return outputs
    
data = scipy.io.loadmat("flower.mat")
X = np.atleast_2d(unwrap(data["X"]))
y = np.atleast_2d(unwrap(data["y"])).astype(np.int32)
y[y==2] = -1

### RBF

In [ ]:
C = 10
sigma = 0.1
options = {'verb': False, 'tmax': float('inf'),
           'kernel': 'rbf', 'sigma': sigma}
 
model = my_kernel_svm(X, y, C, options)
plot_pts(X, y)
plt.axis('equal')
plt.grid(linestyle='--')
plt.title('RBF kernel $\sigma$={}'.format(sigma))
plot_boundary(plt.gca(), model)
plt.savefig('flower_rbf.png')

### Polynomial

In [ ]:
C = 10000
d = 4
options = {'verb': False, 'tmax': float('inf'),
           'kernel': 'polynomial', 'd': d}
 
model = my_kernel_svm(X, y, C, options)
plot_pts(X, y)
plt.axis('equal')
plt.grid(linestyle='--')
plt.title('Polynomial kernel $d$={}'.format(d))
plot_boundary(plt.gca(), model)
plt.savefig('flower_polynomial.png')

# Part 2: Model selection

In [ ]:
data = scipy.io.loadmat("ocr_data_2D_trn.mat")
X_trn = data['X']
y_trn = data['y']

In [ ]:
np.random.seed(42)
trn_folds, tst_folds = crossval(y_trn.size, num_folds=4)
Cs = [0.0001 ,0.001, 0.01, 0.1]
ds = [5, 10, 15]

best_params = None
best_err = np.inf
for C in Cs:
    for d in ds:
        options = {'verb': False, 'tmax': float('inf'),
                  'kernel': 'polynomial', 'd': d}
        err = compute_kernel_test_error(trn_folds, tst_folds, X_trn, y_trn, C, options)

        if err < best_err:
            best_err = err
            best_params = (C, d)
        print('C {}, d {} -> err {}'.format(C, d, err))

print('best params: C={}, d={}'.format(best_params[0], best_params[1]))

In [ ]:
C = best_params[0]
d = best_params[1]

options = {'verb': False, 'tmax': float('inf'),
           'kernel': 'polynomial', 'd': d}
model = my_kernel_svm(X_trn, y_trn, C, options)
plot_pts(X_trn, y_trn)
plt.axis('equal')
plt.grid(linestyle='--')
plt.title('Polynomial kernel $d$={} TRN data'.format(d))
plot_boundary(plt.gca(), model)

In [ ]:
data = scipy.io.loadmat("ocr_data_2D_tst.mat")
X_tst = data['X']
y_tst = data['y']
images_tst = unwrap(data['tst'])[0]

In [ ]:
test_err = "Probably around 42?"
print(test_err)

In [ ]:
plot_pts(X_tst, y_tst)
plt.axis('equal')
plt.grid(linestyle='--')
plt.title('Polynomial kernel $d$={} TST data'.format(d))
plot_boundary(plt.gca(), model)
plt.savefig('ocr_polynomial_kernel_tst.png')

In [ ]:
labels = classif.copy()
labels[labels == 1] = 0
labels[labels == -1] = 1

plt.figure()
show_classification(images_tst, labels, 'AC')
plt.savefig('ocr_svm_classif.png')

# Part 3: Real world example - digit classification

In [ ]:
data = scipy.io.loadmat("mnist_01_trn.mat")
X_trn = data['X']
y_trn = data['y']

In [ ]:
np.random.seed(42)
trn_folds, tst_folds = crossval(y_trn.size, num_folds=5)
Cs = [0.01, 0.1, 1, 10]
sigmas = [0.5, 0.9, 1.5]

best_params = None
best_err = np.inf
for C in Cs:
    for sigma in sigmas:
        options = {'verb': False, 'tmax': float('inf'),
                   'kernel': 'rbf', 'sigma': sigma}
        err = compute_kernel_test_error(trn_folds, tst_folds, X_trn, y_trn, C, options)

        if err < best_err:
            best_err = err
            best_params = (C, sigma)
        print('C {}, sigma {} -> err {}'.format(C, sigma, err))

print('best params: C={}, sigma={}'.format(best_params[0], best_params[1]))

In [ ]:
C = best_params[0]
sigma = best_params[1]

options = {'verb': False, 'tmax': float('inf'),
           'kernel': 'rbf', 'sigma': sigma}
model = my_kernel_svm(X_trn, y_trn, C, options)

In [ ]:
data = scipy.io.loadmat("mnist_01_tst.mat")
X_tst = data['X']
y_tst = data['y']

In [ ]:
classif = ?
err = ?
print(np.round(err, 8))

In [ ]:
print(classif.shape)
show_mnist_classification(X_tst, classif)
plt.savefig('mnist_tst_classif.png')